## Identify hand-drawn digits prototype

### Imports

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Defining datasets/dataloaders and variables

In [ ]:
TRAIN_BATCH_SZ = 64
TEST_BATCH_SZ = 1000
TRAIN_EPOCH = 7
LEARNING_RATE = 0.01


training_set = datasets.MNIST(
    root= "./data",
    train= True,
    download= True,
    transform= ToTensor()
)

test_set = datasets.MNIST(
    root = "./data",
    train= False,
    download= True,
    transform= ToTensor()
)

train_dl = DataLoader(training_set, batch_size= TRAIN_BATCH_SZ)